In [1]:
from os.path import join, expanduser
import numpy as np
from affine import Affine
from rasters import Raster
from emit_tools import load_EMIT_swath, extract_GLT, apply_GLT, ortho_xr
from emit_tools import FILL_VALUE as fill_value
from emit_tools import GLT_NODATA_VALUE as GLT_nodata_value


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/emit_tools/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/emit_tools/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/homebrew/Caskroom/mambaforge/base/envs/emit_tools/lib/python3.11/site-p

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

In [ ]:
filename = expanduser(join("~", "Downloads", "EMIT_L2A_RFL_001_20240521T232728_2414216_003.nc"))

In [ ]:
swath_ds = load_EMIT_swath(filename)
swath_ds

In [ ]:
GLT_array = extract_GLT(swath_ds)
GLT_array

In [ ]:
type(GLT_array)

In [ ]:
GLT_array.shape

In [ ]:
ortho_ds = ortho_xr(swath_ds)
ortho_ds

In [ ]:
swath_array = swath_ds["reflectance"].data
swath_array

In [ ]:
swath_array.shape

In [ ]:
type(swath_array)

In [ ]:
ortho_array = apply_GLT(swath_array, GLT_array)
ortho_array

In [ ]:
type(ortho_array)

In [ ]:
ortho_array.shape

In [ ]:
# Build Output Dataset
if swath_array.ndim == 2:
    swath_array = swath_array[:, :, np.newaxis]

# ortho_array_shape = (GLT_array.shape[0], GLT_array.shape[1], swath_array.shape[-1])

latitude_length = ortho_array.shape[0]
longitude_length = ortho_array.shape[1]
band_length = swath_array.shape[-1]
ortho_array_shape = (latitude_length, longitude_length, band_length)

ortho_array = np.full(
    ortho_array_shape,
    fill_value,
    dtype=np.float32,
)

valid_GLT = np.all(GLT_array != GLT_nodata_value, axis=-1)

# Adjust for One based Index - make a copy to prevent decrementing multiple times inside ortho_xr when applying the glt to elev
glt_array_copy = GLT_array.copy()
glt_array_copy[valid_GLT] -= 1
ortho_array[valid_GLT, :] = swath_array[
    glt_array_copy[valid_GLT, 1], glt_array_copy[valid_GLT, 0], :
]

ortho_array

In [ ]:
ortho_array.shape

In [ ]:
latitude_length = GLT_array.shape[0]
longitude_length = GLT_array.shape[1]
band_length = swath_array.shape[-1]
ortho_array_shape = (latitude_length, longitude_length, band_length)
ortho_array_shape

In [ ]:
swath_ds.geotransform

In [ ]:
type(swath_ds)

In [ ]:
affine = Affine.from_gdal(*swath_ds.geotransform)
affine